In [70]:
#Task 2

import numpy as np
import pandas as pd
import scipy.stats as stats

# Simulation parameters
n_simulations = 10**4  # Number of simulations to approximate the distribution
beta = 10**8  # Inverse temperature
lambda_step = 10**(-4)  # Step size for SGLD
gamma = 10**(-8)  # Regularization parameter
#theta = 0  # Initial guess for theta

# Set quantile levels and number of iterations for SGLD
q = 0.95
n_iterations = 10**6
n_assets = 3  # Number of assets in the portfolio
# Define the initial weights for the portfolio
initial_weights = np.array([1.0 / n_assets] * n_assets)

# Asset parameters for each scenario

asset_params = [
    [{'mu': 500, 'sigma': 1}, {'mu': 0, 'sigma': 10**6}, {'mu': 0, 'sigma': 10**-4}],
    [{'mu': 500, 'sigma': 1}, {'mu': 0, 'sigma': 10**6}, {'mu': 0, 'sigma': 1}],
    [{'mu': 0, 'sigma': 10**3}, {'mu': 0, 'sigma': 1}, {'mu': 0, 'sigma': 4}],
    [{'mu': 0, 'sigma': 1}, {'mu': 1, 'sigma': 4}, {'mu': 0, 'sigma': 10**-4}],
    [{'mu': 0, 'sigma': 1}, {'mu': 1, 'sigma': 4}, {'mu': 2, 'sigma': 1}]
]


In [71]:


#Simulate asset returns
def simulate_asset_returns(params, size):
    return [np.random.normal(asset['mu'], asset['sigma'], size) for asset in params]
print(simulate_asset_returns(asset_params[0],10000))
# Generate asset returns
def simulate_asset_returns_1(mu, sigma, size):
    return np.random.normal(mu, sigma, size)

# Calculate VaR and CVaR from simulated returns
def calculate_var_cvar(returns, q):
    var = np.quantile(returns, q)
    # Sort returns
    sorted_returns = np.sort(returns)
    # Find the index where VaR would be positioned in the sorted list
    var_index = np.searchsorted(sorted_returns, var, side='right')
    # CVaR is the average of the worst 1-q percent of returns
    cvar = np.mean(sorted_returns[var_index:])
    return var, cvar

# Calculate VaR and CVaR from simulated returns
def calculate_var_cvar_1(returns, q):
    var = np.quantile(returns, q)
    # Sort returns
    sorted_returns = np.sort(returns)
    # Find the index where VaR would be positioned in the sorted list
    var_index = np.searchsorted(sorted_returns, var, side='right')
    # CVaR is the average of the worst 1-q percent of returns
    cvar = np.mean(sorted_returns[var_index:])
    return  var
# Calculate VaR and CVaR from simulated returns
def calculate_var_cvar_2(returns, q):
    var = np.quantile(returns, q)
    # Sort returns
    sorted_returns = np.sort(returns)
    # Find the index where VaR would be positioned in the sorted list
    var_index = np.searchsorted(sorted_returns, var, side='right')
    # CVaR is the average of the worst 1-q percent of returns
    cvar = np.mean(sorted_returns[var_index:])
    return  cvar

def V(theta_hat, weights, q, gamma, params):
    X = simulate_asset_returns(params, 1)
    # Calculate the positive part of returns - theta
    portfolio_losses = ((np.sum(np.exp(weights) * X))/ (np.sum(np.exp(weights)))) - theta_hat
    return np.mean(np.maximum(portfolio_losses,0)/(1 - q) + theta_hat) + gamma * np.abs(theta_hat)**2 


def partial_g_w(weights, params, j):
    sum_exp_weights = np.sum(np.exp(weights))
    g_w = np.zeros_like(weights)
    X = np.array(simulate_asset_returns(params, 1))
    for i in range(len(weights)):
        if i != j:
            g_w[i] = -np.sum(X * (np.exp(weights[i]) * np.exp(weights[j])) / sum_exp_weights**2)
        else:
            g_w[i] = np.sum(X * ((sum_exp_weights - np.exp(weights[i])) - np.exp(weights[i])) / sum_exp_weights**2)
    return np.sum(g_w)

def H_theta(theta_hat, weights, params, q, gamma):
    X = np.array(simulate_asset_returns(params, 1))
    portfolio_losses = ((np.sum(np.exp(weights) * X))/ (np.sum(np.exp(weights)))) - theta_hat
    # Filter for positive losses
    positive_losses = portfolio_losses > 0
    # Calculate the average loss where the condition is True
    loss = np.mean(portfolio_losses[positive_losses]) if np.any(positive_losses) else 0
    # Regularization term
    regularization = 2 * gamma * theta_hat**2
    # The complete loss is the sum of the two terms
    return 1 - (1/(1 - q)) * loss + regularization

def H_wj(theta_hat, weights, params,j, q, gamma):
    X = np.array(simulate_asset_returns(params, 1))
    portfolio_losses = ((np.sum(np.exp(weights) * X))/ (np.sum(np.exp(weights)))) - theta_hat
    # Filter for positive losses
    positive_losses = portfolio_losses > 0
    # Calculate the average loss where the condition is True
    loss = np.mean(portfolio_losses[positive_losses]) if np.any(positive_losses) else 0
    return (1/(1 - q)) * partial_g_w(weights, params, j) * loss + 2 * gamma * weights[j]

[array([500.42836796, 499.49379377, 499.45993278, ..., 501.10083044,
       498.424526  , 499.66569343]), array([ -362509.73310171, -1395875.16020609, -1184788.09751686, ...,
        -485762.56052776, -2042933.83435082,  -101307.65994096]), array([-7.50985438e-05, -1.09046429e-04,  1.29986612e-04, ...,
        1.66693109e-04,  3.54178889e-05,  1.03680841e-05])]


In [72]:
# Initialize the minimum CVaR to a large number
min_cvar = float('inf')

# Define a function to find optimal weights for a given scenario
def find_optimal_weights(scenario):
    global min_cvar
    min_cvar = float('inf')
    optimal_weights = None
    # Create a grid of weight combinations to search
    for w1 in np.linspace(0, 1, 20):
        for w2 in np.linspace(0, 1 - w1, 20):
            w3 = 1 - w1 - w2
            returns_X1 = simulate_asset_returns_1(scenario[0]['mu'], scenario[0]['sigma'], n_simulations)
            returns_X2 = simulate_asset_returns_1(scenario[1]['mu'], scenario[1]['sigma'], n_simulations)
            returns_X3 = simulate_asset_returns_1(scenario[2]['mu'], scenario[2]['sigma'], n_simulations)
            portfolio_returns = w1 * returns_X1 + w2 * returns_X2 + w3 * returns_X3
            cvar = calculate_var_cvar_2(portfolio_returns, q)
            if cvar < min_cvar:
                min_cvar = cvar
                optimal_weights = (w1, w2, w3)
    return optimal_weights, min_cvar ,calculate_var_cvar_1(portfolio_returns, q)

# Iterate over each scenario to find the optimal weights and CVaR
optimal_weights_all_scenarios = []
min_cvar_all_scenarios = []
var_all = []

for scenario in asset_params:
    optimal_weights, min_cvar, min_var = find_optimal_weights(scenario)
    optimal_weights_all_scenarios.append(optimal_weights)
    min_cvar_all_scenarios.append(min_cvar)
    var_all.append(min_var)

# Display the results
for i, (weights, var, cvar) in enumerate(zip(optimal_weights_all_scenarios, var_all, min_cvar_all_scenarios)):
    print(f"Scenario {i+1}:")
    print(f"Optimal weights: {weights}")
    print(f"Minimum VaR: {var}")
    print(f"Minimum CVaR: {cvar}")

Scenario 1:
Optimal weights: (0.0, 0.0, 1.0)
Minimum VaR: 501.62836934976275
Minimum CVaR: 0.000206419283677028
Scenario 2:
Optimal weights: (0.0, 0.0, 1.0)
Minimum VaR: 501.6647265862163
Minimum CVaR: 2.077807121129769
Scenario 3:
Optimal weights: (0.0, 0.9473684210526315, 0.052631578947368474)
Minimum VaR: 1640.9717869195256
Minimum CVaR: 1.9679162853010606
Scenario 4:
Optimal weights: (0.0, 0.0, 1.0)
Minimum VaR: 1.65001915951139
Minimum CVaR: 0.000205476218005448
Scenario 5:
Optimal weights: (1.0, 0.0, 0.0)
Minimum VaR: 1.678568982788096
Minimum CVaR: 2.0020663464542436


In [73]:
# The main SGLD optimization routine
def sgld_optimization(params, q, gamma, lambda_step, n_iterations, beta):
    # Initialize theta and weights
    theta = np.zeros(n_assets + 1)
    initial_weights = np.array([1.0 / n_assets] * n_assets)
    for _ in range(n_iterations):
        # portfolio_returns = np.dot(weights, asset_returns)
        returns = simulate_asset_returns(params, n_simulations) 
        #portfolio_returns = calculate_var_cvar(returns, q)
        #portfolio_returns = np.dot(weights, returns)
        # Calculate the gradient for theta and update it
        theta_gradient = H_theta(theta[0], initial_weights, params, q, gamma)
        theta[0] -= lambda_step * theta_gradient + np.sqrt(2 * lambda_step / beta) * np.random.randn()

        # Calculate the gradient for each weight and update them
        for j in range(len(initial_weights)):
            wj_gradient = H_wj(theta[0], initial_weights, params, j, q, gamma)
            #weights[j] -= lambda_step * wj_gradient + np.sqrt(2 * lambda_step / beta) * np.random.randn()
            theta[j+1] -= lambda_step * wj_gradient + np.sqrt(2 * lambda_step / beta) * np.random.randn()
        
        # theta -= lambda_step * (theta_gradient, wj_gradient) + np.sqrt(2 * lambda_step / beta) * np.random.randn()

        # Re-normalize weights after update
        initial_weights /= np.sum(initial_weights)
      
    # Calculate the portfolio's VaR and CVaR using the optimized weights and theta
    VaR_SGLD = theta
    CVaR_SGLD = V(theta, initial_weights, q, gamma, params)  # This would typically use theta to calculate VaR_SGLD, CVaR_SGLD
    
    return initial_weights, VaR_SGLD, CVaR_SGLD

# Run the optimization for each scenario
for scenario_params in asset_params:
    optimized_weights, VaR, CVaR = sgld_optimization(
        scenario_params, q, gamma, lambda_step, n_iterations, beta
    )
    print(f"Optimized weights: {optimized_weights}")
    print(f"Value at Risk (VaR): {VaR}")
    print(f"Conditional Value at Risk (CVaR): {CVaR}")



KeyboardInterrupt: 